<a href="https://colab.research.google.com/github/cannedhedgehog/Saturday/blob/main/Lab_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [53]:
import numpy as np

Функция активации (сигмоида)

In [54]:
def sigmoid(x):
    return 1 / (1 + np.exp(-x))

Производная сигмоиды для обратного распространения

In [55]:
def sigmoid_derivative(x):
    return x * (1 - x)

Этот класс создаёт простую нейросеть с одним скрытым слоем, где можно: задать количество нейронов в скрытом слое, обучить сеть на примере, получать предсказания

In [56]:
class NeuralNetwork:
    def __init__(self, input_size, hidden_size):
        """
        Конструктор класса NeuralNetwork.

        Аргументы:
        input_size  - количество входных нейронов (размер входного вектора).
        hidden_size - количество нейронов в скрытом слое (определяет сложность модели).

        Внутри:
        - Создаются и инициализируются случайные веса для входного и скрытого слоя.
        - Создаются смещения (bias) для скрытого и выходного слоя, инициализированные нулями.
        """

        self.input_size = input_size  # Количество входных признаков
        self.hidden_size = hidden_size  # Количество нейронов в скрытом слое

        # Матрица весов от входного слоя к скрытому (размер input_size x hidden_size)
        self.weights_input_hidden = np.random.randn(input_size, hidden_size)

        # Матрица весов от скрытого слоя к выходному (размер hidden_size x 1)
        self.weights_hidden_output = np.random.randn(hidden_size, 1)

        # Смещения для скрытого слоя (размер 1 x hidden_size, т.к. один вектор смещения для всех примеров)
        self.bias_hidden = np.zeros((1, hidden_size))

        # Смещение для выходного слоя (размер 1 x 1, т.к. всего один выход)
        self.bias_output = np.zeros((1, 1))

    def feedforward(self, X):
        """
        Прямой проход (forward propagation) через сеть.

        Аргумент:
        X - входной массив размерностью (количество примеров x input_size)

        Внутри:
        1. Умножаем входные данные на веса, прибавляем смещение (X * W + b)
        2. Применяем сигмоидную функцию активации для скрытого слоя.
        3. Повторяем процесс для выходного слоя.

        Возвращает:
        - Выходное значение после применения сигмоидной функции.
        """

        # Расчёт взвешенной суммы на скрытом слое: X * W + b
        self.hidden_input = np.dot(X, self.weights_input_hidden) + self.bias_hidden

        # Применяем сигмоидную функцию активации
        self.hidden_output = sigmoid(self.hidden_input)

        # Расчёт взвешенной суммы на выходном слое
        self.final_input = np.dot(self.hidden_output, self.weights_hidden_output) + self.bias_output

        # Применяем сигмоидную функцию активации на выходе
        return sigmoid(self.final_input)

    def train(self, X, y, epochs=1000, lr=0.1):
        """
        Обучение модели с помощью обратного распространения ошибки (Backpropagation).

        Аргументы:
        X      - входные данные (размерность: количество примеров x input_size)
        y      - целевые значения (размерность: количество примеров x 1)
        epochs - количество эпох (сколько раз обновляем веса)
        lr     - скорость обучения (learning rate)

        Внутри:
        1. Выполняется прямой проход.
        2. Вычисляется ошибка между предсказанием и целевыми значениями.
        3. Производится обратное распространение ошибки:
           - Градиент выхода (как сильно нужно скорректировать выход).
           - Градиент скрытого слоя (как сильно скорректировать скрытые нейроны).
        4. Обновляются веса и смещения по методу градиентного спуска.
        """

        for _ in range(epochs):
            # 1. Прямой проход: получаем предсказание сети
            output = self.feedforward(X)

            # 2. Вычисление ошибки (разница между истинным значением y и предсказанным output)
            error = y - output

            # 3. Вычисляем градиент для выходного слоя (производная сигмоиды * ошибка)
            d_output = error * sigmoid_derivative(output)

            # 4. Ошибка скрытого слоя (как сильно скрытый слой влияет на конечный результат)
            error_hidden = d_output.dot(self.weights_hidden_output.T)

            # 5. Градиент скрытого слоя (производная сигмоиды * ошибка скрытого слоя)
            d_hidden = error_hidden * sigmoid_derivative(self.hidden_output)

            # 6. Обновление весов с учётом градиента и скорости обучения:
            #    - Весы между скрытым и выходным слоем
            #    - Весы между входным и скрытым слоем
            self.weights_hidden_output += self.hidden_output.T.dot(d_output) * lr
            self.weights_input_hidden += X.T.dot(d_hidden) * lr

            # 7. Обновление смещений
            self.bias_output += np.sum(d_output, axis=0, keepdims=True) * lr
            self.bias_hidden += np.sum(d_hidden, axis=0, keepdims=True) * lr


In [57]:
# Создаем искусственный табличный датасет
X = np.array([[0, 0], [0, 1], [1, 0], [1, 1]])  # XOR-проблема
y = np.array([[0], [1], [1], [0]])  # Ожидаемый выход

In [58]:
# Создаем и обучаем сеть (меняя значение hidden_size задаем колличество нейронов)
nn = NeuralNetwork(input_size=2, hidden_size=4)
nn.train(X, y, epochs=5000, lr=0.1)

In [59]:
# Проверяем предсказания (Вход: [0 1] -> Выход: [0.98]  (≈ 1))
for i in X:
    print(f"Вход: {i}, Выход: {nn.feedforward(i)}")

Вход: [0 0], Выход: [[0.05602212]]
Вход: [0 1], Выход: [[0.9217298]]
Вход: [1 0], Выход: [[0.91688698]]
Вход: [1 1], Выход: [[0.10003555]]
